In [3]:
import pandas as pd
import pickle
import os
import nltk
import re
#nltk.download('wordnet')
import random
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from tqdm import tqdm_notebook as tqdm

In [10]:
#Update our running master_df_labeled with multiple new files
def batch_merge_into_master(master_file, batch_folder):
  batch_files = os.listdir(batch_folder)
  print(batch_files)
  master_df_labeled = pd.read_excel(master_file,index_col=0)
  for new_file in batch_files:
    print(new_file)
    if re.match('batch_[0-9]*',new_file):
      new_df = pd.read_excel(batch_folder+new_file,index_col=0)
      master_df_labeled.update(new_df.true_pos)
      print_tf_pos(master_df_labeled)
  #master_df_labeled.to_excel(master_file)
  print("please save master_df_labeled!")
  return master_df_labeled

#Print out how many examples are labeled as True or False Positives
def print_tf_pos(master_df_labeled):
  print(f"Count of true positives: {master_df_labeled[master_df_labeled.true_pos==1].shape[0]}")
  print(f"Count of false positives: {master_df_labeled[master_df_labeled.true_pos==0].shape[0]}")
  assert master_df_labeled[master_df_labeled.problematic==0].shape[0]==0

In [27]:
#All excel file paths here, pickle paths are used by other scripts so hardcoded
master_file = "./data/master_df_labeled.xlsx"

batch_folder = "./data/please_label_batch/100/"

batch_labeled = "/Users/echo/Google Drive File Stream/My Drive/2019 Fall/Capstone group/FairFrame Annotations/labeled/"
greats_folder = "/Users/echo/Google Drive File Stream/My Drive/2019 Fall/Capstone group/FairFrame Annotations/greats_batches/"
non_greats_folder = "/Users/echo/Google Drive File Stream/My Drive/2019 Fall/Capstone group/FairFrame Annotations/non_greats_batches/"

# Example Workflow
1. Print current true and false positive counts
2. After labeling, update our running master file (merge_into_master)
3. Print new true and false positive counts to check, also select specific example to check

In [11]:
#Print current true and false positive counts
#master_df_labeled.to_excel("./data/master_df_labeled.xlsx")
master_df_labeled = pd.read_excel("./data/master_df_labeled.xlsx",index_col=0)

print_tf_pos(master_df_labeled)
print(master_df_labeled.shape)

Count of true positives: 4096
Count of false positives: 603
(111029, 6)


In [7]:
master_df_labeled[master_df_labeled.true_pos.isin([1,0,-1])==False]

,review,flagged_word,flagged_index,problematic,lemmatized,true_pos


In [51]:
master_df_labeled.to_excel("./data/master_df_labeled.xlsx")

In [8]:
master_df_labeled = batch_merge_into_master("./data/master_df_labeled.xlsx", batch_labeled)
print_tf_pos(master_df_labeled)
print(master_df_labeled.shape)

['batch_2.xlsx', 'batch_3.xlsx', 'batch_7.xlsx', 'batch_1.xlsx', 'batch_0.xlsx', 'batch_9.xlsx', 'batch_10.xlsx', 'batch_11.xlsx', 'batch_5.xlsx', 'batch_6.xlsx', 'batch_12.xlsx', 'batch_13.xlsx', 'batch_14.xlsx', 'batch_72.xlsx', 'batch_34.xlsx', 'batch_48.xlsx', 'batch_60.xlsx', 'batch_32.xlsx', 'batch_51.xlsx', 'batch_33.xlsx', 'batch_59.xlsx', 'batch_31.xlsx', 'batch_22.xlsx', 'batch_23.xlsx', 'batch_24.xlsx', 'batch_47.xlsx', 'batch_52.xlsx', 'batch_53.xlsx', 'batch_58.xlsx', 'batch_61.xlsx', 'batch_62.xlsx', 'batch_20.xlsx', 'batch_19.xlsx', 'batch_18.xlsx', 'batch_17.xlsx', 'batch_16.xlsx', 'batch_26.xlsx', 'batch_29.xlsx', 'batch_30.xlsx', 'batch_40.xlsx', 'batch_49.xlsx', 'batch_50.xlsx', 'batch_56.xlsx', 'batch_8.xlsx', 'batch_70.xlsx', 'batch_54.xlsx', 'Icon\r']
batch_2.xlsx
Count of true positives: 4047
Count of false positives: 647
batch_3.xlsx
Count of true positives: 4047
Count of false positives: 647
batch_7.xlsx
Count of true positives: 4047
Count of false positives: 6

In [9]:
master_df_labeled.to_excel("./data/master_df_labeled.xlsx")
#pickle.dump(master_df_labeled,open(master_file,"wb"))

# Resampling for less common examples

In [17]:
filter_words = ["great", "outstanding", "excellent", "professional", "creative", "experience"]
batch_nums = range(200,1110)

In [18]:
df_list = []
for b in batch_nums:
  df_list.append(pd.read_excel(batch_folder+"batch_"+str(b)+".xlsx",index_col=0))


In [20]:
concat_df = pd.concat(df_list)
concat_df

,review,flagged_word,flagged_index,true_pos
11274,"['great', 'work', ',', 'professional', ',', 'p...",great,0,-1
79390,"['great', 'job', '!']",great,0,-1
52043,"['as', 'usual', ',', 'we', 'were', 'highly', '...",response,11,-1
105269,"['finally', 'we', 'decided', 'to', 'discontinu...",team,39,-1
100771,"['great', 'working', 'experience', '.']",experience,2,-1
...,...,...,...,...
26900,"['excellent', 'throughout', '.']",excellent,0,-1
57686,"['everything', 'is', 'fine', ',', 'he', 'shoul...",creative,9,-1
53583,"['she', 'nailed', 'the', 'brief', ',', 'and', ...",thorough,10,-1
54773,"['i', 'highly', 'recommend', 'hiring', 'arun',...",skills,22,-1


In [21]:
concat_df["lemmatized"] = [wordnet_lemmatizer.lemmatize(w) for w in concat_df.flagged_word]

In [25]:
greats_df = concat_df[concat_df.lemmatized.isin(filter_words)]
nongreats_df = concat_df[~concat_df.lemmatized.isin(filter_words)]
nongreats_df

,review,flagged_word,flagged_index,true_pos,lemmatized
52043,"['as', 'usual', ',', 'we', 'were', 'highly', '...",response,11,-1,response
105269,"['finally', 'we', 'decided', 'to', 'discontinu...",team,39,-1,team
101644,"['always', 'helpful', ',', 'responsive', 'and'...",helpful,1,-1,helpful
116171,"['order', 'was', 'accepted', 'on', 'the', '24t...",order,0,-1,order
48354,"['understands', 'the', 'need', 'without', 'muc...",able,8,-1,able
...,...,...,...,...,...
52798,"['great', 'job', ',', 'quick', ',', 'reliable'...",reliable,5,-1,reliable
120564,"['mario', 'is', 'a', 'pleasure', 'to', 'work',...",exceptional,9,-1,exceptional
53583,"['she', 'nailed', 'the', 'brief', ',', 'and', ...",thorough,10,-1,thorough
54773,"['i', 'highly', 'recommend', 'hiring', 'arun',...",skills,22,-1,skill


In [33]:
#Create fixed chunks of shuffled data for people to label
def batch_please_label_greats(df, batch_folder, cat, sample_num=100):
  df = df.drop(labels=["lemmatized"],axis=1)
  row_num = df.shape[0]
  batches = row_num//sample_num
  print(batches)
  #assert 1==0
  
  for start_bin in range(batches):
    end_bin = start_bin+1
    temp = df.iloc[start_bin*sample_num:end_bin*sample_num,:]
    if end_bin == batches and row_num%sample_num>0:#we have leftover rows to deal with
      extra = df.iloc[end_bin*sample_num:,:]
      extra.to_excel(batch_folder+"batch_"+str(end_bin)+"_"+cat+".xlsx")
    temp.to_excel(batch_folder+"batch_"+str(start_bin)+"_"+cat+".xlsx")
  print(f"Full Batches: {batches}")
  print(f"Partial Batch?: {row_num%sample_num>0}")
  print(f"Go check {batch_folder}")

In [34]:
batch_please_label_greats(greats_df,greats_folder, "greats")
batch_please_label_greats(nongreats_df,non_greats_folder, "non_greats")

486
Full Batches: 486
Partial Batch?: True
Go check /Users/echo/Google Drive File Stream/My Drive/2019 Fall/Capstone group/FairFrame Annotations/greats_batches/
422
Full Batches: 422
Partial Batch?: True
Go check /Users/echo/Google Drive File Stream/My Drive/2019 Fall/Capstone group/FairFrame Annotations/non_greats_batches/


# Initial Labeling Process

In [33]:
master_df = pickle.load(open("./data/master_df.p", "rb"))
master_df["lemmatized"] = [wordnet_lemmatizer.lemmatize(w) for w in master_df.flagged_word]
#
print(len(set(master_df.flagged_word)))
print(len(set(master_df.lemmatized)))
#print(set(master_df.flagged_word)-set(master_df.lemmatized))
# print(sorted(list(set(master_df.lemmatized)),key=str.lower))
# master_df.shape

524
448


In [8]:
# #creating random sample from master_df to label
# sampled = master_df[master_df.problematic==1].sample(500)
# sampled.to_excel("./data/fptp.xlsx")

In [34]:
#merge by row index
labeled = pd.read_excel("./data/fptp_labeled.xlsx",index_col=0)
master_df_labeled = master_df.join(labeled.true_pos).fillna(-1)
master_df_labeled = master_df_labeled[master_df_labeled.problematic==1]
#master_df_labeled
print_tf_pos(master_df_labeled)



Count of true positives: 84
Count of false positives: 31


In [35]:
#master file to continuously update
master_df_labeled.to_excel("./data/master_df_labeled.xlsx")
pickle.dump(master_df_labeled,open("./data/master_df_labeled.p","wb"))

In [ ]:
#Generates new sample of data to be labeled
def please_label(master_file, new_file, sample_num=100):
  master_df = pd.read_excel(master_file,index_col=0)
  to_label_df = master_df[master_df.true_pos==-1]
  output_df = to_label_df.sample(sample_num)
  output_df.to_excel(new_file)
  return output_df

#Create fixed chunks of shuffled data for people to label
def batch_please_label(master_file, batch_folder, sample_num=100):
  master_df = pd.read_excel(master_file,index_col=0).drop(labels=["lemmatized","problematic"],axis=1)
  to_label_df = master_df[master_df.true_pos==-1].sample(frac=1,random_state=0)
  row_num = to_label_df.shape[0]
  batches = row_num//sample_num
  print(batches)
  #assert 1==0
  
  for start_bin in range(batches):
    end_bin = start_bin+1
    temp = to_label_df.iloc[start_bin*sample_num:end_bin*sample_num,:]
    if end_bin == batches and row_num%sample_num>0:#we have leftover rows to deal with
      extra = to_label_df.iloc[end_bin*sample_num:,:]
      extra.to_excel(batch_folder+"batch_"+str(end_bin)+".xlsx")
    temp.to_excel(batch_folder+"batch_"+str(start_bin)+".xlsx")
  print(f"Full Batches: {batches}")
  print(f"Partial Batch?: {row_num%sample_num>0}")
  print(f"Go check {batch_folder}")

In [ ]:
#Update our running master_df_labeled, main function to produce files for dataloaders
def merge_into_master(master_file, new_file, grounds_file, batch=False):
  master_df_labeled = pd.read_excel(master_file,index_col=0)
  new_df = pd.read_excel(new_file,index_col=0)

  master_df_labeled.update(new_df.true_pos)
  master_df_labeled.to_excel(master_file)
  
  pickle.dump(master_df_labeled,open("./data/master_df_labeled.p","wb"))
  return master_df_labeled
  

#Produces ground_truths_df
def produce_ground_truths(master_df_labeled, grounds_file):
  ground_truths_df = master_df_labeled[master_df_labeled.true_pos!=-1]
  ground_truths_df.to_excel(grounds_file)
  pickle.dump(ground_truths_df,open("./data/ground_truths_df.p","wb"))
  return ground_truths_df